In [1]:
`




/home/travis_s/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
  0%|          | 0/32 [00:00<?, ?it/s]It looks like you're training your model on a CPU. Consider switching to a GPU; otherwise, this method can take hours upon hours or even days to finish. For more information, see https://detecto.readthedocs.io/en/latest/usage/quickstart.html#technical-requirements
Epoch 1 of 12
Begin iterating over training dataset
  0%|          | 0/32 [00:00<?, ?it/s]Loss: 1.353833794593811
Epoch 2 of 12
Begin iterating over training dataset
  0%|          | 0/32 [00:00<?, ?it/s]Loss: 1.2757842242717743
Epoch 3 of 12
Begin iterating over training dataset
  0%|          | 0/32 [00:00<?, ?it/s]Loss

<Figure size 2880x1440 with 0 Axes>

In [2]:
from detecto import core, utils, visualize
import os
import random 
import glob
import matplotlib.pyplot as plt
import cv2
from xml.dom import minidom#, parse
import numpy

from torchvision import transforms
from detecto.utils import normalize_transform
from tensorflow.keras.callbacks import TerminateOnNaN
nan_stopping = TerminateOnNaN()


learning_rates = [.001, .0005, .0001]
step_sizes = [3, 5, 7, 10]


# Histogram equalized 

os.chdir('/home/travis_s/repos/infloresence_detection/final_set')

dataset = core.Dataset('./train/')

loader = core.DataLoader(dataset, batch_size=1, shuffle=True)

val_dataset = core.Dataset('./val/')

model = core.Model(['flower'])

# Grid search hist

for lr in learning_rates:
    for ss in step_sizes:

        new_dir = f"./learning_rate_{str(lr).replace('0.','')}_step_size_{str(ss)}"
        os.mkdir(new_dir)

        losses = model.fit(loader, val_dataset, epochs=12, learning_rate=lr, verbose=True, lr_step_size=ss)
        print(losses)
        if losses == 'nan':
            print('Stopped')
            break
        plt.plot(losses)
        plt.title(f"Faster R-CNN losses learning_rate_{str(lr).replace('0.','')}_step_size_{str(ss)}")
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.savefig(f"./{new_dir}/learning_rate_{str(lr).replace('0.','')}_step_size_{str(ss)}.png", dpi=300, transparent=False, facecolor='w', edgecolor='w')
        plt.close('all')

        model.save(f"./model_weights_infloresence_2021-07-16_12_epoc_w_step_{str(ss)}_lr_{str(lr).replace('0.','')}.pth")
        try:
            import xml.etree.ElementTree as ET

            detect_dict = {}

            for img in glob.glob('./test/*.png'):
                image_name = os.path.basename(img)
                cnt = 0
                image = utils.read_image(img)
                predictions = model.predict(image)

                # predictions format: (labels, boxes, scores)
                labels, boxes, scores = predictions
                print(labels, scores)
                a_img = cv2.imread(img)
                a_img = cv2.cvtColor(a_img, cv2.COLOR_BGR2RGB)
                copy = a_img.copy()

                xml = img.replace('.png', '.xml')
                mydoc = minidom.parse(xml)
                items = mydoc.getElementsByTagName('object')
                tree = ET.parse(xml)
                root = tree.getroot()

                list_with_all_boxes = []

                for roi in root.iter('object'):
                    file_name = root.find('filename').text
                    ymin, xmin, ymax, xmax = None, None, None, None 

                    ymin = int(roi.find("bndbox/ymin").text)
                    xmin = int(roi.find("bndbox/xmin").text)
                    ymax = int(roi.find("bndbox/ymax").text)
                    xmax = int(roi.find("bndbox/xmax").text)
                    gt = [ymin, xmin, ymax, xmax]

                    start_point = (xmin, ymax)
                    end_point = (xmax, ymin)
                    color = (0, 0, 255) 
                    thickness = 6
                    cv2.rectangle(a_img, start_point, end_point, color, thickness)

                for i, box in enumerate(boxes):

                    if scores[i] >= 0.1:
                        cnt += 1
                        min_x, min_y, max_x, max_y = int(box[0]), int(box[1]), int(box[2]), int(box[3])
                        ml = [min_y, min_x, max_y, max_x]

                        start_point = (min_x, max_y)
                        end_point = (max_x, min_y)
                        color = (255, 0, 0) 
                        thickness = 6

                        new_img = copy[min_y:max_y, min_x:max_x, :]
                        cv2.rectangle(a_img, start_point, end_point, color, thickness)

                plt.figure(figsize=(40, 20))
                plt.imsave('f"./{new_dir}/{image_name}')  # display it
        except:
                pass

  0%|          | 0/32 [00:00<?, ?it/s]It looks like you're training your model on a CPU. Consider switching to a GPU; otherwise, this method can take hours upon hours or even days to finish. For more information, see https://detecto.readthedocs.io/en/latest/usage/quickstart.html#technical-requirements
Epoch 1 of 12
Begin iterating over training dataset
  0%|          | 0/32 [00:00<?, ?it/s]Loss: 1.4303413033485413
Epoch 2 of 12
Begin iterating over training dataset
  0%|          | 0/32 [00:00<?, ?it/s]Loss: 1.5494545996189117
Epoch 3 of 12
Begin iterating over training dataset
  0%|          | 0/32 [00:00<?, ?it/s]Loss: 1.5057341754436493
Epoch 4 of 12
Begin iterating over training dataset
  0%|          | 0/32 [00:00<?, ?it/s]Loss: 1.5909144282341003
Epoch 5 of 12
Begin iterating over training dataset
  0%|          | 0/32 [00:00<?, ?it/s]Loss: 1.5670252442359924
Epoch 6 of 12
Begin iterating over training dataset
  0%|          | 0/32 [00:00<?, ?it/s]Loss: 1.5648278594017029
Epoch 7

<Figure size 2880x1440 with 0 Axes>